In [26]:
states = ["Happy", "Sad"]
end_state = 'death'
death_p = 1 / (365*80)

start_probability = {"Happy": 0.6, "Sad": 0.4}
 
transition_probability = {
   "Happy" : {"Happy": 0.7, "Sad": 0.3-death_p, 'death': death_p},
   "Sad" : {"Happy": 0.5, "Sad": 0.5-death_p, 'death': death_p},
   }
 
emission_probability = {
   "Happy" : {"smile": 0.7, "serious": 0.25, "tears": 0.05},
   "Sad" : {"smile": 0.1, "serious": 0.3, "tears": 0.6},
   }

In [38]:
def fwd_bkw(observations, states, start_prob, trans_prob, emm_prob, end_st):
    # Forward part of the algorithm
    fwd = [] # The ALPHAS in the slides
    f_prev = {}
    for i, observation_i in enumerate(observations):
        f_curr = {}
        for st in states:
            if i == 0:
                # If we are in the first observation is the base case we go to the starting probabilities
                prev_f_sum = start_prob[st]
            else:
                # Otherwise is just the sum seen in the slides
                prev_f_sum = sum(f_prev[k]*trans_prob[k][st] for k in states)
                
            # We always multiply by the emission probabilities
            f_curr[st] = emm_prob[st][observation_i] * prev_f_sum
        
        # And we store all the values in a list (to use them "recursively")
        fwd.append(f_curr)
        f_prev = f_curr

    p_fwd = sum(f_curr[k] * trans_prob[k][end_st] for k in states)

    # Backward part of the algorithm
    bkw = [] # The BETAS in the slides
    b_prev = {}
    for i, observation_i_plus in enumerate(reversed(observations[1:]+[None,])):
        b_curr = {}
        for st in states:
            if i == 0:
                # base case for backward part
                b_curr[st] = trans_prob[st][end_st]
            else:
                b_curr[st] = sum(trans_prob[st][l] * emm_prob[l][observation_i_plus] * b_prev[l] for l in states)

        bkw.insert(0,b_curr)
        b_prev = b_curr

    p_bkw = sum(start_prob[l] * emm_prob[l][observations[0]] * b_curr[l] for l in states)

    # Merging the two parts
    posterior = []
    for i in range(len(observations)):
        posterior.append({st: fwd[i][st] * bkw[i][st] / p_fwd for st in states})
    
    return posterior

In [44]:
observations = ["smile", "serious", "tears"]

posteriors = fwd_bkw(observations,
                   states,
                   start_probability,
                   transition_probability,
                   emission_probability,
                   end_state)

for i,p in enumerate(posteriors):
    print ("At step", i, "this are the probabilities of being in each step", p)

At step 0 this are the probabilities of being in each step {'Happy': 0.9027711020617021, 'Sad': 0.0972288979382981}
At step 1 this are the probabilities of being in each step {'Happy': 0.5424856068454804, 'Sad': 0.4575143931545196}
At step 2 this are the probabilities of being in each step {'Happy': 0.12351569140927171, 'Sad': 0.8764843085907283}


In [46]:
new_observations = ["smile", "serious", "tears", "smile", "smile", "smile", "smile"]

posteriors= fwd_bkw(new_observations,
                   states,
                   start_probability,
                   transition_probability,
                   emission_probability,
                   end_state)

for i,p in enumerate(posteriors):
    print ("At step", i, "the person is", new_observations[i],"and these s are the probabilities of being in mental state", p)

At step 0 the person is smile and these s are the probabilities of being in mental state {'Happy': 0.9032548915725614, 'Sad': 0.09674510842743858}
At step 1 the person is serious and these s are the probabilities of being in mental state {'Happy': 0.549085967776482, 'Sad': 0.45091403222351806}
At step 2 the person is tears and these s are the probabilities of being in mental state {'Happy': 0.157039817043986, 'Sad': 0.8429601829560142}
At step 3 the person is smile and these s are the probabilities of being in mental state {'Happy': 0.9108420644755258, 'Sad': 0.08915793552447425}
At step 4 the person is smile and these s are the probabilities of being in mental state {'Happy': 0.9509518385208339, 'Sad': 0.049048161479166366}
At step 5 the person is smile and these s are the probabilities of being in mental state {'Happy': 0.9523773514123793, 'Sad': 0.04762264858762087}
At step 6 the person is smile and these s are the probabilities of being in mental state {'Happy': 0.9391085889164629,